# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from data_wrangling import combine_info_utils, base_processing_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import warnings

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca
import quantities as pq

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

## exp

In [ ]:
raw_data_folder_path = 'all_monkey_data/raw_monkey_data/monkey_Bruno/data_0226'
print('='*100)
print(raw_data_folder_path)

# try:    
# check if the files already exist (hardcoded for now)
neural_path = raw_data_folder_path.replace('raw_monkey_data', 'planning_and_neural')
df1_path = os.path.join(neural_path, 'event_aligned/time_resolved_cv_scores_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv')
df2_path = os.path.join(neural_path, 'event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv')
if os.path.exists(df1_path) and os.path.exists(df2_path):
    print('Files already exist')

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
time_resolved_regression.streamline_getting_time_resolved_cv_scores(pn, cv_folds=2)

## Iterate over session

In [ ]:
pn.time_resolved_cv_scores_gpfa = pn.all_scores_df
pn.time_resolved_cv_scores_gpfa['bin_mid_time'] = pn.time_resolved_cv_scores_gpfa['new_bin'] * \
    pn.bin_width + pn.new_bin_start_time + pn.bin_width/2

pn.time_resolved_cv_scores_gpfa['trial_count'] = pn.time_resolved_cv_scores_gpfa['train_trial_count']

pn.plot_time_resolved_regression(time_resolved_cv_scores=pn.time_resolved_cv_scores_gpfa, score_threshold_to_plot=0.1)

In [ ]:
raw_data_dir_name = 'all_monkey_data/raw_monkey_data'

for monkey_name in ['monkey_Bruno', 'monkey_Schro']:
    sessions_df_for_one_monkey = combine_info_utils.make_sessions_df_for_one_monkey(
            raw_data_dir_name, monkey_name)

    # with warnings.catch_warnings():
    #     warnings.simplefilter("ignore")
    for index, row in sessions_df_for_one_monkey.iterrows():
        if row['finished'] is True:
            continue
        raw_data_folder_path = os.path.join(
            raw_data_dir_name, row['monkey_name'], row['data_name'])
        print('='*100)
        print(raw_data_folder_path)
        
        # try:    
        # check if the files already exist (hardcoded for now)
        neural_path = raw_data_folder_path.replace('raw_monkey_data', 'planning_and_neural')
        df1_path = os.path.join(neural_path, 'event_aligned/time_resolved_cv_scores_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv')
        df2_path = os.path.join(neural_path, 'event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv')
        if os.path.exists(df1_path) and os.path.exists(df2_path):
            print('Files already exist')
            continue
        
        pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
        time_resolved_regression.streamline_getting_time_resolved_cv_scores(pn)
        sessions_df_for_one_monkey.loc[index, 'finished'] = True
        # except Exception as e:
        #     print(e)
        #     continue
        



## visualize

In [ ]:
pn.time_resolved_cv_scores

In [ ]:
pn.time_resolved_cv_scores_gpfa

In [ ]:
pn.time_resolved_cv_scores

In [ ]:
raw_data_dir_name = 'all_monkey_data/raw_monkey_data'
monkey_name = 'monkey_Bruno'
sessions_df_for_one_monkey = combine_info_utils.make_sessions_df_for_one_monkey(
        raw_data_dir_name, monkey_name)


for feature in ['time_rel_to_stop', 'cur_ff_distance', 'cur_ff_rel_x', 'nxt_ff_distance', 'nxt_ff_rel_x', 'time_since_last_capture']:
    print('='*100)
    print('='*100)
    print(feature)
    for index, row in sessions_df_for_one_monkey.iterrows():
        if row['finished'] is True:
            continue
        raw_data_folder_path = os.path.join(
            raw_data_dir_name, row['monkey_name'], row['data_name'])
        print('='*100)
        print(raw_data_folder_path)
        
        # check if the files already exist (hardcoded for now)
        neural_path = raw_data_folder_path.replace('raw_monkey_data', 'planning_and_neural')
        df1_path = os.path.join(neural_path, 'event_aligned/time_resolved_cv_scores_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv')
        df2_path = os.path.join(neural_path, 'event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv')
        if not (os.path.exists(df1_path) and os.path.exists(df2_path)):
            print('Files don\'t exist yet.')
            continue
        
        pn.time_resolved_cv_scores_gpfa = pd.read_csv(df1_path)
        pn.time_resolved_cv_scores = pd.read_csv(df2_path)
        
        new_cv_scores = pd.concat([pn.time_resolved_cv_scores[['new_bin', 'bin_mid_time', 'trial_count', feature]], 
                                pn.time_resolved_cv_scores_gpfa[[feature]].rename(columns={feature: f'{feature}_cv_w_gpfa'})], axis=1)
        pn.plot_time_resolved_regression(time_resolved_cv_scores=new_cv_scores, score_threshold_to_plot=None,
                                        rank_by_max_score=False)


        break